In [2]:
from bs4 import BeautifulSoup
import pickle

with open('data/ChatLog_Chime_1_6_2022.html', mode='r', encoding="utf8") as fd:
    html_doc = fd.read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [3]:
messages = soup.find_all('div', 'message')

print(len(messages))

5500


In [4]:
def is_generic_roll(message_class_list):
    return True if 'rollresult' in message_class_list else False
    
def is_new_character_rolling(message):
    return message.find(class_='spacer')

In [5]:
name_dictionary = {
    'Jason W. (GM)' : 'NPC',
    'Jason W.' : 'NPC',
    'NPC' : 'NPC',
    'Mayor' : 'NPC',
    'Kevin L.' : 'Russel',
    'Sally C.' : 'Vye',
    'vye' : 'Vye',
    'Vye' : 'Vye',
    'Larissa J.' : 'Roll',
    'Roll' : 'Roll',
    'Elaina C.' : 'Wing',
    'Wing' : 'Wing',
    'Eva Y.' : 'Monti',
    'Monti' : 'Monti',
    'Montineau' : 'Monti',
    'MontiNO' : 'Monti',
    '(To Yev)' : 'Monti',
    'Brian T.' : 'Dragonborn',
    'DergenBern' : 'Dragonborn',
    'The Dragonborn' : 'Dragonborn',
    'Vicky L.' : 'June/Auggie',
    'Auggie' : 'June/Auggie',
    'Yev' : 'Yev',
    'Genki' : 'Genki',
    'Don C.' : 'Tithe',
    'Tithe' : 'Tithe',
    'Rachel W.' : 'Sebille',
    'Sebille' : 'Sebille'
}
my_character_name = 'Monti'

roll_dictionary = {
    'ATHLETICS' : 'Ability Check',
    'ACROBATICS' : 'Ability Check',
    'SLEIGHT OF HAND' : 'Ability Check',
    'STEALTH' : 'Ability Check',
    'ARCANA' : 'Ability Check',
    'HISTORY' : 'Ability Check',
    'INSIGHT' : 'Ability Check',
    'PERFORMANCE' : 'Ability Check',
    'DECEPTION' : 'Ability Check',
    'PERSUASION' : 'Ability Check',
    'ANIMAL HANDLING' : 'Ability Check',
    'MEDICINE' : 'Ability Check',
    'SURVIVAL' : 'Ability Check',
    'PERCEPTION' : 'Ability Check',
    'INTIMATION' : 'Ability Check',
    'RELIGION' : 'Ability Check',
    'INVESTIGATION' : 'Ability Check',
    'NATURE' : 'Ability Check',
    'STRENGTH' : 'Ability Check',
    'DEXTERITY' : 'Ability Check',
    'CONSTITUTION' : 'Ability Check',
    'INTELLIGENCE' : 'Ability Check',
    'WISDOM' : 'Ability Check',
    'CHARISMA' : 'Ability Check',
    'INITIATIVE' : 'Initiative'  
}

In [38]:
from dateutil.parser import parse

current_time = None
timestamps = []
for message in messages:
    tstamp = message.find(class_='tstamp')
    if tstamp:
        current_time = parse(tstamp.string)
    timestamps.append(current_time)

print(len(timestamps))
print(timestamps[-20:])

5500
[datetime.datetime(2021, 7, 4, 0, 42), datetime.datetime(2021, 7, 4, 0, 42), datetime.datetime(2021, 7, 4, 0, 42), datetime.datetime(2021, 7, 4, 0, 43), datetime.datetime(2021, 7, 4, 0, 43), datetime.datetime(2021, 7, 4, 0, 43), datetime.datetime(2021, 7, 4, 0, 43), datetime.datetime(2021, 7, 4, 0, 43), datetime.datetime(2021, 7, 4, 0, 44), datetime.datetime(2021, 7, 4, 0, 44), datetime.datetime(2021, 7, 4, 0, 45), datetime.datetime(2021, 7, 4, 0, 45), datetime.datetime(2021, 7, 4, 0, 46), datetime.datetime(2021, 7, 4, 0, 46), datetime.datetime(2021, 7, 4, 0, 48), datetime.datetime(2021, 7, 4, 0, 48), datetime.datetime(2021, 7, 4, 0, 48), datetime.datetime(2021, 7, 4, 0, 50), datetime.datetime(2021, 7, 4, 0, 56), datetime.datetime(2021, 7, 4, 0, 56)]


In [39]:
has_roll = []
for message in messages:
    has_roll.append((message.find(class_='inlinerollresult') is not None) or ('rollresult' in message['class']))

print(len([i for i in has_roll if i ]))

roll_msgs = [message for i, message, in zip(has_roll, messages) if i ]

1717


In [63]:
import re
characters = [] # Who rolled
roll_types = [] # What kind of roll: (1) Ability Check (2) Saving Throw (3) Initiative (4) Spell/Attack (5) Generic
d20_rolls = [] # Filter rolls made by 1d20 rolls only

roll_re = re.compile(r'">([0-9]+)</span>')

roll_timestamps = []

current_char = ''
    
for message in roll_msgs:    
    is_generic = is_generic_roll(message['class'])
    
    if is_generic:
        # Who
        character = current_char
        name_character = current_char
        
        if message.find(class_='spacer'):
            character = message.find(class_='by').get_text().strip()[:-1]
            name_character = name_dictionary[character] # Human readable
        current_char = character # Update

        # Filter 1d20 only; Amount
        if 'rolling d20' in message.find(class_='formula').get_text():
            amnt = int(message.find(class_='rolled').get_text())
            
            # Append
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
                roll_timestamps.append(current_time)
            else:
                continue
            characters.append(name_character)
            roll_types.append('Generic')
            d20_rolls.append(amnt)
        
    else:
        character = ''
        
        # Who
        if message.find(class_='spacer'):
            current_char = message.find(class_='by').get_text().strip()[:-1]
            character = name_dictionary[current_char]
        else:
            character = name_dictionary[current_char]
            
        # What type; Amount
        inline_span = message.find('span', class_='inlinerollresult')
        roll_txt = ''
        if 'original-title' in inline_span.attrs:
            roll_txt = inline_span['original-title']
        else:
            roll_txt = inline_span['title']
        if 'Rolling 1d20' in roll_txt:
            amnt = int(roll_re.search(roll_txt).group(1))
            
            sheet_label = message.find(class_='sheet-label')
            if not sheet_label:
                continue
            roll_type_txt = sheet_label.find('span').get_text().split('(')[0].strip().upper()
            roll_type = ''
            
            if roll_type_txt in roll_dictionary:
                roll_type = roll_dictionary[roll_type_txt]
            elif 'SAVE' in roll_type_txt:
                roll_type = 'Saving Throw'
            else:
                roll_type = 'Spell/Attack'
            
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
                roll_timestamps.append(current_time)
            else:
                continue
            characters.append(character)
            roll_types.append(roll_type)
            d20_rolls.append(amnt)
            

print('Characters:', len(characters), characters)
print('Roll Types:', len(roll_types), roll_types)
print('d20 rolls:', len(d20_rolls), d20_rolls)
print('Timestamps:', len(roll_timestamps), roll_timestamps)

Characters: 596 ['Monti', 'Russel', 'Russel', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Monti', 'Roll', 'Monti', 'Roll', 'Monti', 'Roll', 'Monti', 'Monti', 'Roll', 'Tithe', 'NPC', 'Dragonborn', 'NPC', 'Yev', 'Roll', 'Yev', 'Dragonborn', 'NPC', 'Monti', 'NPC', 'Yev', 'Dragonborn', 'Vye', 'Yev', 'Tithe', 'NPC', 'Monti', 'Roll', 'Vye', 'Vye', 'Yev', 'Vye', 'Dragonborn', 'Yev', 'Yev', 'NPC', 'Yev', 'Monti', 'NPC', 'Yev', 'Dragonborn', 'Tithe', 'Roll', 'Wing', 'Monti', 'Monti', 'Yev', 'Dragonborn', 'June/Auggie', 'Monti', 'Dragonborn', 'Vye', 'Sebille', 'Vye', 'Dragonborn', 'Roll', 'Sebille', 'June/Auggie', 'Tithe', 'Vye', 'Sebille', 'Tithe', 'Sebille', 'Sebille', 'Tithe', 'Tithe', 'NPC', 'Tithe', 'June/Auggie', 'Tithe', 'Yev', 'Tithe', 'Yev', 'Tithe', 'NPC', 'Tithe', 'June/Auggie', 'Monti', 'Tithe', 'Tithe', 'Tithe', 'Monti', 'Sebille', 'Dragonborn', 'Wing', 'Sebille', 'Tithe', 'Sebille', 'Yev', 'Vye', 'Monti', 'Yev', 'Vye', 'Vye', 'Wing', '

In [64]:
from datetime import timedelta, datetime

MIN_ROLLS = 20
MAX_GAP = timedelta(hours=12)

start_time = roll_timestamps[0]
last_time = roll_timestamps[0]
count = 1

sessions = []

session_idx = []

for time in roll_timestamps:
    session_idx.append(len(sessions))
    print(time, session_idx[-1], count)
    if time - last_time > MAX_GAP:
        print('GAP', count)
        if count > MIN_ROLLS:
            print('COUNT')
            sessions.append((start_time, last_time, count))
            print("Added session:", last_time, count)
        count = 1
        start_time = time
    else:
        count += 1
    last_time = time
if count > MIN_ROLLS:
    sessions.append((start_time, last_time, count))
    print(last_time-start_time, count)

print('Sessions:', len(sessions), sessions)
print('Session Idx:', session_idx)
    
with open('data/chime_sessions.txt', 'w') as fd:
    for session in sessions:
        fd.write(session[0].strftime('%Y-%m-%d\n'))

2018-10-14 13:26:00 0 1
2018-10-14 13:34:00 0 2
2018-10-14 13:34:00 0 3
2018-10-14 13:36:00 0 4
2018-10-14 13:52:00 0 5
2018-10-14 14:04:00 0 6
2018-10-14 14:07:00 0 7
2018-10-14 14:14:00 0 8
2018-10-14 14:45:00 0 9
2018-10-14 15:28:00 0 10
2018-10-14 15:29:00 0 11
2018-10-14 17:26:00 0 12
2018-10-14 17:37:00 0 13
2018-10-14 18:03:00 0 14
2018-10-14 18:10:00 0 15
2018-10-14 18:18:00 0 16
2018-10-14 18:32:00 0 17
2018-10-14 18:43:00 0 18
2018-10-14 18:53:00 0 19
2018-10-14 19:02:00 0 20
2020-04-18 20:26:00 0 21
GAP 21
COUNT
Added session: 2018-10-14 19:02:00 21
2020-04-18 20:38:00 1 1
2020-04-18 20:39:00 1 2
2020-04-18 21:10:00 1 3
2020-04-18 21:10:00 1 4
2020-04-18 21:11:00 1 5
2020-04-18 21:36:00 1 6
2020-04-18 21:37:00 1 7
2020-04-18 22:03:00 1 8
2020-04-18 22:14:00 1 9
2020-04-18 22:15:00 1 10
2020-04-18 22:21:00 1 11
2020-04-18 22:45:00 1 12
2020-04-18 22:52:00 1 13
2020-04-18 22:56:00 1 14
2020-04-18 23:44:00 1 15
2020-05-23 20:17:00 1 16
GAP 16
2020-05-23 20:20:00 1 1
2020-05-23 

In [65]:
import pandas as pd

roll_df = pd.DataFrame({
    'time': roll_timestamps,
    'character': characters,
    'session': session_idx,
    'type': roll_types,
    'value': d20_rolls
})

roll_df = roll_df.dropna()

roll_df.to_csv('data/chime_rolls.zip', index=False)

print(len(roll_df['character'].unique()))
print(len(roll_df['type'].unique()))
print(len(roll_df['session'].unique()))

print(roll_df['character'].unique())

12
5
12
['Monti' 'Russel' 'Roll' 'Tithe' 'NPC' 'Dragonborn' 'Yev' 'Vye' 'Wing'
 'June/Auggie' 'Sebille' 'Genki']


In [66]:
roll_df = pd.read_csv('data/chime_rolls.zip', parse_dates=['time'])

print(roll_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       596 non-null    datetime64[ns]
 1   character  596 non-null    object        
 2   session    596 non-null    int64         
 3   type       596 non-null    object        
 4   value      596 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 23.4+ KB
None


In [69]:
import plotly.express as px

last_session_df = roll_df[roll_df['time'] > datetime(2021,5,28)]
print(len(last_session_df))

monti_df =  last_session_df[last_session_df['character'] == 'Monti']
print(monti_df)

fig = px.histogram(last_session_df, x="value")
fig.show()

49
                   time character  session           type  value
559 2021-05-29 21:04:00     Monti       10     Initiative     15
570 2021-05-29 22:46:00     Monti       10  Ability Check      7


In [70]:
counts = roll_df.groupby(['character']).count()['time'].sort_values()
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(counts)

character
Russel           2
Genki           16
NPC             18
Wing            20
June/Auggie     36
Sebille         43
Tithe           44
Dragonborn      56
Roll            65
Yev             80
Monti           88
Vye            128
Name: time, dtype: int64


In [71]:
counts = roll_df.groupby(['type']).count()['time'].sort_values()
print(counts)

type
Saving Throw      52
Initiative        71
Spell/Attack     111
Generic          174
Ability Check    188
Name: time, dtype: int64


In [72]:
data = roll_df.groupby(['session','type']).count()['value'].reset_index(level=[0,1])
print(data)

    session           type  value
0         0        Generic     21
1         1  Ability Check     37
2         1        Generic     38
3         1     Initiative     12
4         1   Saving Throw      5
5         1   Spell/Attack     24
6         2  Ability Check      6
7         2        Generic      9
8         2     Initiative      5
9         2   Spell/Attack      6
10        3  Ability Check      5
11        3        Generic     26
12        3     Initiative      4
13        3   Saving Throw      1
14        3   Spell/Attack     17
15        4  Ability Check     13
16        4        Generic     10
17        4   Saving Throw      1
18        4   Spell/Attack      5
19        5  Ability Check     34
20        5        Generic     24
21        5     Initiative     14
22        5   Saving Throw     23
23        5   Spell/Attack     27
24        6  Ability Check     16
25        6        Generic     10
26        6     Initiative      1
27        6   Spell/Attack      7
28        7  A

In [73]:
print(sum(roll_df['character'] == 'Monti'))

88
